# Chains in LangChain

## Outline
- LLMChain
- Sequential Chains
    - SimpleSequentialChain
    - SequentialChain
- Router Chain

In [25]:
import warnings
warnings.filterwarnings('ignore')


In [26]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [4]:
#!pip install pandas

In [27]:
import pandas as pd
df = pd.read_csv('Data.csv')


In [28]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [29]:
from langchain.chat_models import ChatOpenAI # model
from langchain.prompts import ChatPromptTemplate # prompt
from langchain.chains import LLMChain # chain


In [30]:
llm = ChatOpenAI(temperature=0.9)


In [31]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)


In [32]:
chain = LLMChain(llm=llm, prompt=prompt)


In [33]:
product = "Queen Size Sheet Set"
chain.run(product)


'RoyalRest'

In [39]:
llm = ChatOpenAI(temperature=0.9)


In [40]:
prompt = ChatPromptTemplate.from_template(
    "描述一家製造{product}的公司，最好的名稱是什麼?"
)


In [41]:
chain = LLMChain(llm=llm, prompt=prompt)


In [42]:
product = "皇后尺寸床單組"
chain.run(product)


KeyboardInterrupt: 

## SimpleSequentialChain
Single Input Single Output

In [34]:
from langchain.chains import SimpleSequentialChain


In [35]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [36]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)


In [37]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )


In [38]:
overall_simple_chain.run(product)




> Entering new SimpleSequentialChain chain...
Royal Dreams
Royal Dreams is a luxury travel agency offering personalized and exclusive vacations for discerning clients seeking unforgettable experiences.

> Finished chain.


'Royal Dreams is a luxury travel agency offering personalized and exclusive vacations for discerning clients seeking unforgettable experiences.'

In [43]:
from langchain.chains import SimpleSequentialChain


In [44]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "描述一家製造{product}的公司，最好的名稱是什麼?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [45]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "為以下公司撰寫20字的描述：{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)


In [46]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )


In [47]:
overall_simple_chain.run(product)




> Entering new SimpleSequentialChain chain...
這家製造皇后尺寸床單組的公司可以命名為「Royal Linens」。
「Royal Linens」是一家製造皇后尺寸床單組的公司。

> Finished chain.


'「Royal Linens」是一家製造皇后尺寸床單組的公司。'

## SequentialChain
Multiple Input Multiple Output

In [15]:
from langchain.chains import SequentialChain


In [16]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [17]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [18]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [19]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [20]:
# overall_chain: input= Review 
# and output= English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)


In [21]:
df.Review[5]


"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [22]:
review = df.Review[5]
overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': "The reviewer finds the taste of the product mediocre, and suspects that it might be an old batch or counterfeit since the foam doesn't hold and the store-bought ones taste much better.",
 'followup_message': "Réponse : Nous sommes désolés d'apprendre que notre produit n'a pas répondu à vos attentes en termes de goût. Nous tenons à vous assurer que nous mettons tout en œuvre afin de nous assurer de la qualité de nos produits. Cependant, votre commentaire est précieux pour nous, car il nous aide à identifier et à résoudre tout problème éventuel. Nous vous invitons à nous contacter directement avec plus de détails sur le produit e

In [49]:
from langchain.chains import SequentialChain


In [50]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "將以下評論翻譯成繁體中文："
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="Chinese_Review"
                    )


In [51]:
second_prompt = ChatPromptTemplate.from_template(
    "你能用一句話總結以下評論嗎："
    "\n\n{Chinese_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [52]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "以下評論是什麼語言：\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [53]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "針對以下內容撰寫後續回應 "
    "用指定語言的摘要："
    "\n\n摘要: {summary}\n\n語言: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [54]:
# overall_chain: input= Review 
# and output= English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["Chinese_Review", "summary","followup_message"],
    verbose=True
)


In [55]:
df.Review[5]


"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [56]:
review = df.Review[5]
overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'Chinese_Review': '我覺得它的味道一般般。泡沫不持久，很奇怪。我在商店買的同款產品口味明顯好得多...\n是舊批次還是假貨!?',
 'summary': '這產品的味道一般，泡沫不持久，與商店買的同款相比明顯差很多，讓我懷疑是舊批次或假貨。',
 'followup_message': "Je suis désolé d'apprendre que vous avez été déçu(e) par notre produit. La qualité de nos produits est très importante pour nous et nous prenons vos commentaires au sérieux. Nous nous excusons si vous avez reçu un produit de mauvaise qualité. Pourriez-vous nous fournir plus de détails sur votre achat afin que nous puissions enquêter et résoudre ce problème pour vous ? Nous ferons tout notre possible pour rectifier la situation et vous assurer une expérience satisfaisante avec notre marque. Merci de nous avoir fait part de votre préoccupation."}

## Router Chain

In [23]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""


In [24]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]


In [25]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate


In [26]:
llm = ChatOpenAI(temperature=0)


In [27]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)


In [28]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)


In [29]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""


In [30]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)


In [31]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )


In [32]:
chain.run("What is black body radiation?")




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an object that absorbs all incident radiation and reflects or transmits none. It is called "black body" because it absorbs all wavelengths of light, appearing black at room temperature. \n\nAccording to Planck\'s law, black body radiation is characterized by a continuous spectrum of wavelengths and intensities, which depend on the temperature of the object. As the temperature increases, the peak intensity of the radiation shifts to shorter wavelengths, resulting in a change in color from red to orange, yellow, white, and eventually blue at very high temperatures.\n\nBlack body radiation is a fundamental concept in physics and has significant applications in various fields, including astrophysics, thermodynamics, and quantum mechanics. It played a crucial role in the development of quantum theory and understanding the behavior of light and matter.'

In [33]:
chain.run("what is 2 + 2")




> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for your kind words! As a mathematician, I am happy to help with any math questions, no matter how simple or complex they may be.\n\nThe question you\'ve asked is a basic addition problem: "What is 2 + 2?" To solve this, we can simply add the two numbers together:\n\n2 + 2 = 4\n\nTherefore, the answer to the question "What is 2 + 2?" is 4.'

In [34]:
chain.run("Why does every cell in our body contain DNA?")




> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information necessary for the synthesis of proteins, which are essential for the structure and function of cells. It serves as a blueprint for the production of specific proteins that determine the characteristics and traits of an organism. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next during reproduction. Therefore, every cell in our body contains DNA to ensure the proper functioning and continuity of life.'

In [57]:
physics_template = """你是一位非常聰明的物理學教授。
你擅長以簡潔和容易理解的方式回答物理學問題。
當你不知道問題的答案時，你會承認你不知道。

這裡有一個問題：
{input}"""


math_template = """你是一位非常出色的數學家。
你擅長回答數學問題。
你之所以出色，是因為你能夠將困難的問題分解成各個部分，
回答這些部分，然後將它們組合起來以回答更廣泛的問題。

這裡有一個問題：
{input}"""

history_template = """你是一位非常出色的歷史學家。
你對不同歷史時期的人物、事件和背景具有出色的知識和理解。
你具有思考、反思、辯論、討論和評估過去的能力。你尊重歷史證據，
並能夠利用它來支持你的解釋和判斷。

這裡有一個問題：
{input}"""


computerscience_template = """ 你是一位成功的電腦科學家。
你對創造力、協作、前瞻思考、自信、強大的問題解決能力、理論和演算法的理解，以及出色的溝通技巧充滿熱情。
你擅長回答編程問題。
你之所以出色，是因為你知道如何通過描述機器可以輕易解釋的指令步驟來解決問題，\
並且你知道如何選擇在時間複雜度和空間複雜度之間取得良好平衡的解決方案。

這裡有一個問題：
{input}"""


In [58]:
prompt_infos = [
    {
        "name": "物理", 
        "description": "擅長回答物理學問題", 
        "prompt_template": physics_template
    },
    {
        "name": "數學", 
        "description": "擅長回答數學問題", 
        "prompt_template": math_template
    },
    {
        "name": "歷史", 
        "description": "擅長回答歷史問題", 
        "prompt_template": history_template
    },
    {
        "name": "電腦科學", 
        "description": "擅長回答電腦科學問題", 
        "prompt_template": computerscience_template
    }
]


In [59]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate


In [60]:
llm = ChatOpenAI(temperature=0)


In [61]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)


In [62]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)


In [69]:
MULTI_PROMPT_ROUTER_TEMPLATE = """給定一個原始文本輸入到語言模型中，選擇最適合輸入的模型提示。
您將獲得可用提示的名稱以及提示最適合的描述。
如果您認為修訂它最終將導致語言模型更好的回應，您也可以修改原始輸入。

<< FORMATTING >>
返回一個使用 JSON 物件格式化的 markdown 代碼片段，使其看起來像：
```json
{{{{
    "destination": 字串 \ 使用的提示名稱或「DEFAULT」
    "next_inputs": 字串 \ 原始輸入的可能修改版本
}}}}
```

REMEMBER："destination" 必須是下面指定的候選提示名稱之一，或者如果輸入不適合任何候選提示，則可以是 "DEFAULT"。
REMEMBER："next_inputs" 可以只是原始輸入，如果你認為不需要任何修改的話。

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""


In [70]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)


In [71]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )


In [72]:
chain.run("什麼是黑體輻射？")




> Entering new MultiPromptChain chain...
物理: {'input': '什麼是黑體輻射？'}
> Finished chain.


'黑體輻射是指一個理想化的物體，它能夠完全吸收所有進入它的輻射能量，並以熱能的形式重新發射出來。這種輻射能量的分布與物體的溫度有關，並且在不同波長範圍內的輻射能量也不同。\n\n黑體輻射是物理學中一個重要的概念，它在研究熱力學、量子力學和宇宙學等領域中都有廣泛的應用。根據黑體輻射的研究，我們可以了解到許多物體的熱性質，並且也對宇宙中的星體輻射提供了重要的理論基礎。\n\n希望這個簡單的解釋能夠幫助你理解什麼是黑體輻射。如果你有更多關於這個主題的問題，請隨時提出，我會盡力回答。'

In [73]:
chain.run("2 + 2 是多少？")




> Entering new MultiPromptChain chain...
數學: {'input': '2 + 2 是多少？'}
> Finished chain.


'2 + 2 等於 4。'

In [74]:
chain.run("為什麼我們身體中的每個細胞都含有 DNA？")




> Entering new MultiPromptChain chain...
物理: {'input': '為什麼我們身體中的每個細胞都含有 DNA？'}
> Finished chain.


'非常感謝您對我的能力的認可！關於您的問題，為什麼我們身體中的每個細胞都含有DNA，這是因為DNA是生物體遺傳信息的載體。\n\nDNA是由四種不同的核苷酸組成的長鏈，這些核苷酸的排列順序形成了基因。基因是生物體中控制遺傳特徵的基本單位。每個細胞都需要DNA，因為它包含了生物體的全部遺傳信息。\n\n當細胞分裂時，DNA會複製自身，使得每個新細胞都擁有完整的遺傳信息。這樣，每個細胞都能夠執行其特定的功能，並且能夠傳遞遺傳信息給下一代。\n\n總結來說，每個細胞都含有DNA，是因為DNA是生物體遺傳信息的載體，它包含了控制生物體特徵的基因。希望這個解釋對您有所幫助！如果您有任何其他問題，請隨時提出。'